In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import os


# ------------- Generate Synthetic Data -------------
n_rows = 50000
regions = ['Aceh', 'Tanjung Pinang', 'Jambi', 'Bengkulu', 'Pangkal Pinang', 'Medan', 'Padang', 'Pekanbaru', 'Palembang', 'Bandar Lampung']
start_time = datetime(2025, 1, 1)


timestamps = [start_time + timedelta(hours=i) for i in range(n_rows)]


data = {
    "timestamp": timestamps,
    "meter_id": [f"MTR{random.randint(1000,9999)}" for _ in range(n_rows)],
    "region": [random.choice(regions) for _ in range(n_rows)],
    "voltage": np.round(np.random.normal(220, 5, n_rows), 2),
    "current": np.round(np.random.uniform(0.5, 30.0, n_rows), 2),
    "power_consumption": np.round(np.random.uniform(0.1, 5.0, n_rows), 2),
    "power_factor": np.round(np.random.uniform(0.7, 1.0, n_rows), 2),
    "outage_flag": [random.choice([True, False]) for _ in range(n_rows)],
    "temperature": np.round(np.random.uniform(24, 35, n_rows), 1),
    "humidity": np.round(np.random.uniform(60, 95, n_rows), 1),
    "rainfall": np.round(np.random.exponential(1.5, n_rows), 2),
    "weather_condition": [random.choice(["Cerah", "Berawan", "Hujan ringan", "Hujan deras"]) for _ in range(n_rows)],
    "wind_speed": np.round(np.random.uniform(2, 20, n_rows), 2),
    "population_density": np.round(np.random.uniform(100, 5000, n_rows), 1),
    "urbanization_level": [random.choice(["Perkotaan", "Pedesaan"]) for _ in range(n_rows)],
    "average_income": np.round(np.random.normal(4000000, 1000000, n_rows), 0),
    "economic_activity": [random.choice(["Industri", "Pertanian", "Jasa"]) for _ in range(n_rows)],
    "household_size": np.round(np.random.uniform(2, 6, n_rows), 1),
}


df_bronze = pd.DataFrame(data)


# ------------- Save to Bronze Layer (CSV) -------------
os.makedirs("bronze", exist_ok=True)
df_bronze.to_csv("bronze/data_listrik_raw.csv", index=False)
print("✅ Dataset berhasil dibuat dan disimpan sebagai 'data_listrik_raw.csv'")


✅ Dataset berhasil dibuat dan disimpan sebagai 'data_listrik_raw.csv'


In [2]:
# Load dari Bronze
df = pd.read_csv("bronze/data_listrik_raw.csv", parse_dates=["timestamp"])


# Cleaning contoh:
df_cleaned = df.copy()
df_cleaned["voltage"] = df_cleaned["voltage"].clip(lower=180, upper=240)
df_cleaned["power_factor"] = df_cleaned["power_factor"].clip(0.0, 1.0)


# Drop baris yang hilang kritikal (optional)
df_cleaned.dropna(subset=["power_consumption", "region", "timestamp"], inplace=True)


# Save ke Silver Layer (Parquet)
os.makedirs("silver", exist_ok=True)
df_cleaned.to_parquet("silver/data_listrik_cleaned.parquet", index=False)


In [3]:
# Load dari Silver
df_silver = pd.read_parquet("silver/data_listrik_cleaned.parquet")


# Buat kolom hari atau bulan untuk agregasi
df_silver["date"] = df_silver["timestamp"].dt.date
df_silver["month"] = df_silver["timestamp"].dt.to_period("M").astype(str)


# Agregasi konsumsi listrik rata-rata dan total per wilayah per bulan
df_gold = df_silver.groupby(["region", "month"]).agg({
    "power_consumption": ["sum", "mean"],
    "temperature": "mean",
    "rainfall": "sum",
    "outage_flag": "sum"
}).reset_index()


# Rename kolom multiindex
df_gold.columns = ["region", "month", "total_kwh", "avg_kwh", "avg_temp", "total_rain", "total_outages"]


# Save ke Gold Layer
os.makedirs("gold", exist_ok=True)
df_gold.to_parquet("gold/konsumsi_listrik_aggregated.parquet", index=False)
